Importing libraries and initialising global constants. This code is only valid for two non identical S wave states, as such any matrix elements are not general and should not be copied mindlessly to other codes.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.special import gamma, factorial

STARTING_RANGE_PARAMETER = 1 # In [fm^-2]
ENDING_RANGE_PARAMETER = 15
REDUCED_MASS = 935 * 10/11 # In [Mev / c^2], need to update value and units (10/11 A in MeV)
SUM_LIMIT = 14 # Determines the number of gaussians we expand our wave function to
CORE_MS_RADIUS = 2.30**2 # In fm^2, taken from p.232 of Tanihata et. al. (2013)

V_LS = 21.0 # In MeV
DIFFUSIVITY = 0.6 # Diffusivity, may want to check the vaidity of this paticular number
r_0 = 1.2 # In fm, may want to chose a better value for small nuclei
A_C = 10 # The number of nucleons in the core
SIGMA = 2 #fm
TWO_PARTICLE_POTENTIAL_DEPTH = -0.1 # MeV

CENTRAL_POTENTIAL_PARAMETERS = [0.1, 0.151991, 0.231013, 0.351119, 0.53367, 0.811131, 1.23285,
                                1.87382, 2.84804, 4.32876, 6.57933, 10.]

CENTRAL_MIXING_COEFFICIENTS = [0.0558247,0.214443,2.42773,-0.724055,-2.17761,1.02031,0.819031,-0.96538,0.197094,0.3221,-0.296652,0.093208]

SPIN_ORBIT_POTENTIAL_PARAMETERS = [4.16493, 2.843, 1.941, 1.325, 0.905, 0.618, 0.422, 0.288, 0.196, 0.134, 0.0916, 0.0625]

SPIN_ORBIT_MIXING_COEFFICIENTS = [0.273, -1.307, 3.305, -5.657, 6.565, -3.368, -2.437, 2.638, 0.719, 0.235, 0.0186, 0.00108]

In [2]:
def overlap_matrix_element(range_parameter_i, range_parameter_k, range_parameter_j, range_parameter_l):
    term_1 = ((range_parameter_i * range_parameter_k) / (range_parameter_i**2 + range_parameter_k**2))**1.5
    term_2 = ((range_parameter_j * range_parameter_l) / (range_parameter_j**2 + range_parameter_l**2))**1.5
    return 8 * term_1 * term_2

def kinetic_matrix_element(range_parameter_i, range_parameter_k, μ=REDUCED_MASS, orb_ang_momentum=0):
    term_1 = range_parameter_i * range_parameter_k
    term_2 = range_parameter_i**2 + range_parameter_k**2
    return (197**2 / (2 * μ)) * 12 * np.sqrt(2) * term_1**(1.5) / term_2**(2.5)
    """
    term_1 = 2 * orb_ang_momentum + 3
    term_2 = (range_parameter_i * range_parameter_k)**(-3.5 - orb_ang_momentum)
    term_3 = ((2 * range_parameter_i**2 * range_parameter_k**2) / (range_parameter_i**2 + range_parameter_k**2))**(2.5 + orb_ang_momentum)

    return (197**2 / (2 * μ)) * term_1 * term_2 * term_3
    """

def potential_matrix_element(range_parameter_i, range_parameter_k, central_potential_mixing_coefficient,
                             central_potential_param, orb_ang_momentum=0):
    V_0 = -11.405 * (-1)**0 - 51.175 # Defines V_0 for odd and even l states, shifted from values in capel et. al.
    term_1 = range_parameter_i * range_parameter_k
    term_2 = (1 / range_parameter_i**2) + (1 / range_parameter_k**2) + central_potential_param
    return (2 * np.sqrt(2) * central_potential_mixing_coefficient * V_0) / ((term_1 * term_2)**(1.5))
    """
    V_0 = -11.405 * (-1)**orb_ang_momentum - 51.175 # Defines V_0 for odd and even l states, shifted from values in capel et. al.
    central_term = central_potential_mixing_coefficient * V_0 * (
        central_potential_param + range_parameter_i**(-2) + range_parameter_k**(-2))**(-1.5 - orb_ang_momentum)

    spin_orbit_potential_term = 0
    return (2 / (range_parameter_i * range_parameter_k))**(1.5 + orb_ang_momentum) * (central_term - spin_orbit_potential_term)
    """

def one_particle_overlap_element(range_parameter_i, range_parameter_k):
    return ((2 * range_parameter_i * range_parameter_k) / (range_parameter_i**2 + range_parameter_k**2))**(1.5)

def interaction_matrix_element(range_parameter_i, range_parameter_k, range_parameter_j, range_parameter_l, potential_parameter,
                              potential_strength=TWO_PARTICLE_POTENTIAL_DEPTH):
    alpha_1_square = (range_parameter_i**2 * range_parameter_k**2) / (range_parameter_i**(2) + range_parameter_k**(2))
    alpha_2_square = (range_parameter_j**2 * range_parameter_l**2) / (range_parameter_j**(2) + range_parameter_l**(2))
    term_1 = range_parameter_i * range_parameter_j * range_parameter_k * range_parameter_l
    term_2 = alpha_1_square + alpha_2_square + potential_parameter**2
    return potential_strength * (term_1 / term_2)**1.5

In [3]:
def two_particle_basis_matrix_generation(central_mixing_coefficients=CENTRAL_MIXING_COEFFICIENTS,
                                         central_potential_parameters=CENTRAL_POTENTIAL_PARAMETERS,
                                         size=SUM_LIMIT, potential_parameter=SIGMA):

    first_h_matrix = np.zeros(shape=(size**2, size**2))
    second_h_matrix = np.zeros(shape=(size**2, size**2))
    interaction_matrix = np.zeros(shape=(size**2, size**2))
    overlap_matrix = np.zeros(shape=(size**2, size**2))

    for I in range(1, size**2 + 1):
        i = np.mod((I - 1), size) + 1
        i_range_parameter = next_range_parameter(i - 1)
        j = np.floor((I - 1) / size) + 1
        j_range_parameter = next_range_parameter(j - 1)
        for J in range(1, size**2 + 1):
            k = np.mod((J - 1), size) + 1
            k_range_parameter = next_range_parameter(k - 1)
            l = np.floor((J - 1) / size) + 1
            l_range_parameter = next_range_parameter(l - 1)

            overlap_matrix[I - 1, J - 1] = overlap_matrix_element(i_range_parameter, k_range_parameter,
                                                    j_range_parameter, l_range_parameter)

            interaction_matrix[I - 1, J - 1] = interaction_matrix_element(i_range_parameter, k_range_parameter,
                                                    j_range_parameter, l_range_parameter, potential_parameter)

            first_potential_energy_term = 0
            first_kinetic_term = kinetic_matrix_element(i_range_parameter, k_range_parameter)
            for m in range(len(central_mixing_coefficients)):
                first_potential_energy_term += potential_matrix_element(
                    i_range_parameter, k_range_parameter, central_mixing_coefficients[m],
                    central_potential_parameters[m])
            first_overlap_term = one_particle_overlap_element(j_range_parameter, l_range_parameter)
            first_h_matrix[I - 1, J - 1] = (first_kinetic_term + first_potential_energy_term) * first_overlap_term

            second_potential_energy_term = 0
            second_kinetic_term = kinetic_matrix_element(j_range_parameter, l_range_parameter)
            for m in range(len(central_mixing_coefficients)):
                second_potential_energy_term += potential_matrix_element(
                    j_range_parameter, l_range_parameter, central_mixing_coefficients[m],
                    central_potential_parameters[m])
            second_overlap_term = one_particle_overlap_element(i_range_parameter, k_range_parameter)
            second_h_matrix[I - 1, J - 1] = (second_kinetic_term + second_potential_energy_term) * second_overlap_term

    np.savetxt('nmatrix.csv', overlap_matrix)
    np.savetxt('h1matrix.csv', first_h_matrix)
    np.savetxt('h2matrix.csv', second_h_matrix)
    np.savetxt('interaction.csv', interaction_matrix)
    
    return first_h_matrix + second_h_matrix + interaction_matrix, overlap_matrix


def one_particle_matrix_generation(central_mixing_coefficients=CENTRAL_MIXING_COEFFICIENTS,
                      central_potential_parameters=CENTRAL_POTENTIAL_PARAMETERS,
                      spin_orbit_potential_mixing_coefficients=SPIN_ORBIT_MIXING_COEFFICIENTS,
                      spin_orbit_potential_parameters=SPIN_ORBIT_POTENTIAL_PARAMETERS, size=SUM_LIMIT):
    h_matrix = np.zeros(shape=(size, size))
    n_matrix = np.zeros(shape=(size, size))

    for i in range(size):
        i_range_parameter = next_range_parameter(i)
        for j in range(size):
            j_range_parameter = next_range_parameter(j)
            kinetic_energy_term = kinetic_matrix_element(i_range_parameter, j_range_parameter)
            potential_energy_term = 0
            for k in range(len(central_mixing_coefficients)):
                potential_energy_term += potential_matrix_element(i_range_parameter, j_range_parameter,
                                                                  central_mixing_coefficients[k],
                                                                  central_potential_parameters[k])
            h_matrix[i, j] = kinetic_energy_term + potential_energy_term
            # h_matrix[j, i] = h_matrix[i, j]
            n_matrix[i, j] = one_particle_overlap_element(i_range_parameter, j_range_parameter)
            # ((2 * 10**(np.abs(i - j))) / (1 + 10**(2 * np.abs(i - j))))**(1.5 + orb_ang_momentum)
            # n_matrix[j, i] = n_matrix[i, j]
            # j += 1


    return h_matrix, n_matrix


def next_range_parameter(i, starting_range_parameter=STARTING_RANGE_PARAMETER, ending_range_parameter=ENDING_RANGE_PARAMETER,
                         sum_limit=SUM_LIMIT):
    """
    Finds the next range parameter given the previous and initial range parameters.
    Currently using a simple geometric series to determine range parameters.
    Chose geometric basis parameters $\alpha_i = \alpha_1a^{i-1}$ with initial parameters $\alpha_1 = 0.01, a=2$

    Parameters
    ----------
    i : int detailing the iteration number

    Returns
    -------
    new_range_parameter: float

    """
    geometric_progression_number = (ending_range_parameter / starting_range_parameter)**(1 / (sum_limit - 1))
    new_range_parameter = starting_range_parameter * geometric_progression_number**(i)

    return new_range_parameter

In [4]:
h_matrix, n_matrix = two_particle_basis_matrix_generation()

s_eigenvalues, s_eigenvectors = scipy.linalg.eigh(h_matrix, n_matrix)
#bound_states = np.real(s_eigenvalues) < 0
#print(np.sort(s_eigenvalues[np.real(s_eigenvalues) < 0]))
s_overlap_eigenvalues, s_overlap_eigenvectors = scipy.linalg.eigh(n_matrix)
s_overlap_matrix_condition_number = np.max(s_overlap_eigenvalues) / np.min(s_overlap_eigenvalues)
print(f"The s 1/2 overlap matrix condition number is", s_overlap_matrix_condition_number)

s0_eigenvector = np.asmatrix(s_eigenvectors[:, 0])
s1_eigenvector = np.asmatrix(s_eigenvectors[:, 1])
print("The S state eigenvalues are", s_eigenvalues)
#print(f'The bound states have energies {bound_states}')
#print("The S1 eigenvector is", s1_eigenvector)

The s 1/2 overlap matrix condition number is 6320676719419879.0
The S state eigenvalues are [-1.31395981e+03 -4.69815803e+02 -3.77715802e+02 -3.04295528e+02
 -2.48081760e+02 -2.04230430e+02 -1.55975737e+02 -1.11888847e+02
 -7.00607877e+01 -5.57928528e+01 -3.40063733e+01 -3.21560750e+01
 -3.20976950e+01 -3.09996690e+01 -3.05667477e+01 -2.95211830e+01
 -2.83172678e+01 -2.80895583e+01 -2.46339119e+01 -2.44529057e+01
 -1.87075415e+01 -1.86838794e+01 -1.44534914e+01 -1.21685995e+01
 -1.05925165e+01 -9.16040096e+00 -9.11746988e+00 -2.64821509e+00
 -2.24479592e-01  1.47207659e+00  1.85377194e+00  2.27739327e+00
  3.47159937e+00  3.87958758e+00  4.32036050e+00  5.40146405e+00
  5.92390232e+00  6.20439947e+00  6.23969941e+00  7.03505727e+00
  7.10038510e+00  7.58979681e+00  8.09179952e+00  8.41330276e+00
  1.00317397e+01  1.02619819e+01  1.14604893e+01  1.22397286e+01
  1.28301917e+01  1.33979455e+01  1.38286853e+01  1.46402984e+01
  1.46404498e+01  1.61629404e+01  1.62204337e+01  1.66027555e+0

In [5]:
one_particle_h_matrix, one_particle_n_matrix = one_particle_matrix_generation()

s_eigenvalues, s_eigenvectors = scipy.linalg.eigh(one_particle_h_matrix, one_particle_n_matrix)
s_overlap_eigenvalues, s_overlap_eigenvectors = scipy.linalg.eigh(n_matrix)
s_overlap_matrix_condition_number = np.max(s_overlap_eigenvalues) / np.min(s_overlap_eigenvalues)
print(f"The s 1/2 overlap matrix condition number is", s_overlap_matrix_condition_number)

s0_eigenvector = np.asmatrix(s_eigenvectors[:, 0])
s1_eigenvector = np.asmatrix(s_eigenvectors[:, 1])
print("The S state eigenvalues are", s_eigenvalues)
print("The S1 eigenvector is", s1_eigenvector)

The s 1/2 overlap matrix condition number is 6320676719419879.0
The S state eigenvalues are [-3.27750921e+01 -5.04443693e-01  2.57497590e-01  1.01390263e+00
  2.37675910e+00  4.63354259e+00  8.29598940e+00  1.42120060e+01
  2.37573461e+01  3.90976096e+01  6.40093951e+01  1.04162759e+02
  1.71868943e+02  3.03131100e+02]
The S1 eigenvector is [[ 0.02633071 -0.21504956  0.60569788  0.24533146  0.06597339 -0.30935517
   0.19150912 -0.65147656  0.57684141 -1.11868836  0.97321521 -1.25027631
   0.7099428  -0.47118202]]
